Ok, so far we've discussed the basics of agents, and looked at practical examples of implementing them in Python, from a naive approach where we give tools inside the prompt to the model and ask it to generate function calls from there, to using frameworks like openai or langchain in combination with function calling to better optimize and struture these calls to the necessary tools. 

Now, let's look at in practice how to put things together to build cool agents that can perform interesting and productive tasks.

The framework we'll use to do that is [LangChain](https://python.langchain.com/docs/get_started/introduction).

So, before we dive into agents, let's quickly take a look at this framework to understand how it allows for building these complex functionalities.

In [1]:
# !pip install openai
# !pip install "langchain[all]"
# !pip install docarray
# !pip install ipydrawio
# !pip install ipydrawio_widgets

# Introduction to LangChain 

Working with LLMs involves in one way or another working with a specific type of abstraction: "Prompts".

However, in the practical context of day-to-day tasks we expect LLMs to perform, these prompts won't be some static and dead type of abstraction. Instead we'll work with dynamic prompts re-usable prompts.

# Lanchain

[LangChain](https://python.langchain.com/docs/get_started/introduction.html) is a framework that allows you to connect LLMs together by allowing you to work with modular components like prompt templates and chains giving you immense flexibility in creating tailored solutions powered by the capabilities of large language models.


Its main features are:
- **Components**: abstractions for working with LMs
- **Off-the-shelf chains**: assembly of components for accomplishing certain higher-level tasks

LangChain facilitates the creation of complex pipelines that leverage the connection of components like chains, prompt templates, output parsers and others to compose intricate pipelines that give you everything you need to solve a wide variety of tasks.

At the core of LangChain, we have the following elements:

- Models
- Prompts
- Output parsers

**Models**

Models are nothing more than abstractions over the LLM APIs like the ChatGPT API.​

In [2]:
from langchain.llms import OpenAI
from langchain_openai.chat_models import ChatOpenAI
import os

# os.environ["OPENAI_API_KEY"]=""
chat_model = ChatOpenAI(api_key=os.getenv("OPENAI_API_KEY"), model="gpt-3.5-turbo-1106")

You can predict outputs from both LLMs and ChatModels:

In [3]:
chat_model.invoke("hi!")
# Output: "Hi"

AIMessage(content='Hello! How can I help you today?')

Basic components are:

- Models
- Prompt tempaltes
- Output parsers

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser

In [5]:
prompt = ChatPromptTemplate.from_template("Show me 5 examples of this concept: {concept}")

In [6]:
prompt

ChatPromptTemplate(input_variables=['concept'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['concept'], template='Show me 5 examples of this concept: {concept}'))])

In [7]:
type(prompt)

langchain_core.prompts.chat.ChatPromptTemplate

In [8]:
prompt.format(concept="animal")

'Human: Show me 5 examples of this concept: animal'

In [9]:
chain = prompt | chat_model

output = chain.invoke({"concept": "animal"})
output

AIMessage(content='1. Lion - The king of the jungle, known for its majestic mane and powerful roar.\n2. Eagle - A powerful bird of prey with keen eyesight and impressive hunting skills.\n3. Dolphin - Highly intelligent marine mammals known for their playful behavior and acrobatic displays.\n4. Elephant - The largest land animal, known for its strength, intelligence, and social bonds within herds.\n5. Tiger - A fearsome predator with distinctive orange fur and black stripes, known for its stealth and agility.')

In [10]:
from IPython.display import Markdown

Markdown(output.content)

1. Lion - The king of the jungle, known for its majestic mane and powerful roar.
2. Eagle - A powerful bird of prey with keen eyesight and impressive hunting skills.
3. Dolphin - Highly intelligent marine mammals known for their playful behavior and acrobatic displays.
4. Elephant - The largest land animal, known for its strength, intelligence, and social bonds within herds.
5. Tiger - A fearsome predator with distinctive orange fur and black stripes, known for its stealth and agility.

In [11]:
chat_model.invoke("I am teaching a live-training about LLMs!")

AIMessage(content="That's great! LLMs, or Language Learning Management Systems, are an important tool for language educators and learners. During your live training, be sure to cover the benefits of using LLMs, how to effectively integrate them into language learning curriculum, and demonstrate some of the key features and functionalities of popular LLMs. Additionally, provide practical tips and best practices for maximizing the use of LLMs in language learning. Consider incorporating interactive activities and Q&A sessions to keep participants engaged and encourage active learning. Good luck with your training!")

Finally, you can use the `predict_messages` method over a list of messages:

In [12]:
from langchain.schema import HumanMessage

text = "What would be a good dog name for a dog that loves to nap?"
messages = [HumanMessage(content=text)]

chat_model.invoke(messages)

AIMessage(content='Snooze')

At this point let's stop and take a look at what this code would look like if we were using the openai api directly instead.

Let's understand what is going on.

Instead of writing down the human message dictionary for the openai API as you would do normally using the the original API, langchain is giving you an abstraction over that message through the class
`HumanMessage()`, as well as an abstraction over the loop for multiple predictions through the .`invoke()` method.

Now, why is that an useful thing?

Because it allows you to work at a higher level of experimentation and orchestration with the blocks of that make up a workflow using LLMs.

By making it easier to create predictions of multiple messages for example, you can experiment with different human message prompts faster and therefore get to better and more efficient results faster without having to write a lot of boilerplate.

**Prompts**

The same works for prompts. Now, prompts are pieces of text we feed to LLMs, and LangChain allows you to work with prompt templates.

Prompt Templates are useful abstractions for reusing prompts and they are used to provide context for the specific task that the language model needs to complete. 

A simple example is a `PromptTemplate` that formats a string into a prompt:

In [13]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")
prompt.format(activity="sleeping")
# Output: "What is a good dog name for a dog that loves to nap?"

'What is a good dog name for a dog that loves to sleeping?'

**Output Parsers**

OutputParsers convert the raw output from an LLM into a format that can be used downstream. Here is an example of an OutputParser that converts a comma-separated list into a list:

In [14]:
from langchain.schema import BaseOutputParser

class CommaSeparatedListOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return text.strip().split(", ")

CommaSeparatedListOutputParser().parse("hi, bye")
# Output: ['hi', 'bye']

['hi', 'bye']

This chain will take input variables, pass those to a prompt template to create a prompt, pass the prompt to an LLM, and then pass the output through an output parser.

Ok, so these are the basics of langchain. But how can we leverage these abstraction capabilities inside our LLM app application?

Now, to put everything together LangChain allows you to build something called "chains", which are components that connect prompts, llms and output parsers into a building block that allows you to create more interesting and complex functionality.

Let's look at the example below:

In [15]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good dog name for a dog that loves to {activity}?")

chain = LLMChain(
    llm=ChatOpenAI(),
    prompt=prompt,
)
chain.invoke("sleep")

{'activity': 'sleep',
 'text': 'A good dog name for a dog that loves to sleep could be "Snuggles," "Snoozy," "Dreamer," "Napper," "Sleepy," "Cozy," or "Slumber."'}

So, what the chain is doing is connecting these basic components (the LLM and the prompt template) into
a block that can be run separately. The chain allows you to turn workflows using LLLMs into this modular process of composing components.

Now, the newer versions of LangChain have a new representation language to create these chains (and more) known as LCEL or LangChain expression language, which is a declarative way to easily compose chains together. The same example as above expressed in this LCEL format would be:

In [16]:
chain = prompt | ChatOpenAI()

chain.invoke({"activity": "sleep"})

AIMessage(content='Some good dog names for a dog that loves to sleep could be:\n\n1. Snuggles\n2. Dreamer\n3. Dozer\n4. Napper\n5. Snooze\n6. Slumber\n7. Lazy\n8. Rest\n9. Serene\n10. Zen')

Notice that now the output is an `AIMessage()` object, which represents LangChain's way to abstract the output from an LLM model like ChatGPT or others.

These building blocks and abstractions that LangChain provides are what makes this library so unique, because it gives you the tools you didn't know you need it to build awesome stuff powered by LLMs.

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from IPython.display import Markdown


model = ChatOpenAI(temperature=0)
prompt = ChatPromptTemplate.from_template(template="Name 5 concepts related to this: {concept}. The output should be in bullet points.")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

Markdown(chain.invoke({"concept": "probability distribution"}))

/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


- Discrete probability distribution: This concept refers to a probability distribution that is defined for discrete random variables, where the possible outcomes are countable and distinct.
- Continuous probability distribution: This concept refers to a probability distribution that is defined for continuous random variables, where the possible outcomes form an interval or a range of values.
- Mean (expected value): This concept represents the average value of a random variable in a probability distribution, calculated by summing the products of each possible outcome and its corresponding probability.
- Variance: This concept measures the spread or dispersion of a probability distribution, indicating how far the values of the random variable deviate from the mean.
- Normal distribution: This concept represents a specific type of continuous probability distribution that is symmetric and bell-shaped, commonly used in statistical analysis and modeling.

# LangChain Lab Exercises

Let's take a look at a few examples using some of the capabilities of the LangChain library.

Making a reporting agent.

Let's build an LLM agent that can build simple reports for tasks performed just by reading through the code and files inside a folder + notes from Notion page or .txt file.

First, let's organize our thoughts, an agent will be composed of:

- Tool
- Base LLM
- Conditions for stoping and returning an output 

In [1]:
from langchain_openai.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [2]:
llm.invoke("Hi! Tell me a joke about an instructor who is so childish, he makes examples where he calls himself the agent-master")

AIMessage(content='Why did the instructor become a secret agent? Because he wanted to be the "agent-master" of his own imagination!')

In [3]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)

tools = [get_word_length]

In [4]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [5]:
from langchain.tools.render import format_tool_to_openai_function

llm_with_tools = llm.bind(
    functions=[format_tool_to_openai_function(t) for t in tools]
)

In [6]:
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser


agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps'])
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()

In [7]:
agent.invoke({
    "input": "how many letters in the word educa?",
    "intermediate_steps": []
})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'educa'}, log="\nInvoking: `get_word_length` with `{'word': 'educa'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "educa"\n}', 'name': 'get_word_length'}})])

In [8]:
from langchain.schema.agent import AgentFinish

intermediate_steps = []
while True:
    output = agent.invoke({
        "input": "how many letters in the word educa?",
        "intermediate_steps": intermediate_steps
    })
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(output.tool, output.tool_input)
        tool = {
            "get_word_length": get_word_length
        }[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

get_word_length {'word': 'educa'}
There are 5 letters in the word "educa".


In [9]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
agent_executor.invoke({"input": "how many letters in the word educa?"})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


{'input': 'how many letters in the word educa?',
 'output': 'There are 5 letters in the word "educa".'}

In [27]:
from langchain.prompts import MessagesPlaceholder

MEMORY_KEY = "chat_history"
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are very powerful assistant, but bad at calculating lengths of words."),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

In [28]:
from langchain.schema.messages import HumanMessage, AIMessage
chat_history = []

In [29]:
agent = {
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_functions(x['intermediate_steps']),
    "chat_history": lambda x: x["chat_history"]
} | prompt | llm_with_tools | OpenAIFunctionsAgentOutputParser()
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [30]:
input1 = "how many letters in the word educa?"
result = agent_executor.invoke({"input": input1, "chat_history": chat_history})
chat_history.append(HumanMessage(content=input1))
chat_history.append(AIMessage(content=result['output']))
agent_executor.invoke({"input": "is that a real word?", "chat_history": chat_history})



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'educa'}`


5There are 5 letters in the word "educa".

> Finished chain.


> Entering new AgentExecutor chain...
No, "educa" is not a real word in English.

> Finished chain.


{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'No, "educa" is not a real word in English.'}

In [31]:
agent_executor.invoke({"input": "How many letters in the sentence 'Hi my name is Budha Master'", "chat_history": chat_history})




> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Hi my name is Budha Master'}`


26There are 26 letters in the sentence "Hi my name is Budha Master".

> Finished chain.


{'input': "How many letters in the sentence 'Hi my name is Budha Master'",
 'chat_history': [HumanMessage(content='how many letters in the word educa?'),
  AIMessage(content='There are 5 letters in the word "educa".')],
 'output': 'There are 26 letters in the sentence "Hi my name is Budha Master".'}